In [62]:
import numpy as np
import csv
import urllib.request
import seaborn as sns
import pandas as pd
import json
import re
import os
import glob
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
import nltk
import ssl
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk import word_tokenize,sent_tokenize

In [ ]:
!unzip /content/drive/MyDrive/europarl.zip


In [ ]:
def tokenize_only(text):
  tokens = [word.lower() for word in nltk.word_tokenize(text, language='french')]
  filtered_tokens = []
  for token in tokens:
    #print(token)
    if re.search('[a-zA-Z]', token):
      '''if token not in stopwords.words("french"):'''
      filtered_tokens.append(token)
      
  return filtered_tokens

In [63]:
print(total)

21152667


In [ ]:
print(len(index["indique"]))
print(len(index["européenne"]))
print(len(index["toto"]))

274
363
2


In [ ]:
f = open("index.json", "r")
index = json.load(f)
f.close()

Fonction pour l'affichage des contextes

In [ ]:
def afficher_contextes(chaine , terme , taille_contexte = 30):
  match = re.search(terme , chaine)
  contexts = []
  while match is not None:
      #Les bo rn es gauche et d r o i t e autou r du mot :
      gauche = max(match.start() - taille_contexte - 1, 0)
      droite = match.end()+1+ taille_contexte
      contexts.append(chaine[gauche:droite])
      chaine = chaine[match.end():]
      match = re.search(terme , chaine)
  for c in contexts:
      print(c)

In [ ]:
afficher_contextes(index, "pied", 30)

In [ ]:
path = "europarl/fr"

#Fonction de création de l'index

In [ ]:
def creer_index(path):
  index = { }
  total=0

  tokens=[]
  for chemin in glob.glob(path +'/*.txt'):#[:5]:
    #chaine = lire_fichier(chemin)
    #print(chemin)
    with open(chemin,  encoding="utf8") as file:
    #file_words = file.read().split()
      mots = file.read()
      #content = ' '.join(mots)
      #print(content)
      #print("#", content)
      tokens = tokenize_only(mots)
      total += len(tokens)
    #mots = decouper_mots(chaine)
    for mot in tokens:
      if mot not in index:
        index[mot] = set() #pour ne pas avoir de doublons
      index[mot].add(chemin)


  index = {mot:list(liste_fichiers) for mot, liste_fichiers in index.items()}
  w = open("index.json", "w")
  w.write(json.dumps(index))
  w.close()




#Fonction de création de l'index inversé

In [83]:
# definition de la fonction qui crée l'index inverse
def creer_index_inverse (path) :
  index_inv = { }

  tokens=[]
  for chemin in glob.glob(path +'/*.txt'):#[:5]:
    
    with open(chemin,  encoding="utf8") as file:
    #file_words = file.read().split()
      mots = file.read()
      mots= ' '.join(mots)
      mots= tokenize_only(mots)
      index_inv[chemin] = list()
      index_inv[chemin].append(mots)
    

  index_inv = {chemin:list(liste_mots) for chemin, liste_mots in index_inv.items()}
  w = open("index_inv.json", "w")
  w.write(json.dumps(index_inv))
  w.close()


In [84]:
creer_index_inverse(path)

Lecture du fichier index_inv.json

In [ ]:
f = open("index_inv.json", "r")
index_inv = json.load(f)
f.close()

In [79]:
#test
print(index_inv['europarl/fr/ep-03-10-09-fr.txt'])

['\n\n  \n    \n    \n      \n    \n    \n      \n      \n        &#160;&#160; - L’ordre du jour appelle les déclarations du haut-représentant pour la politique étrangère et de sécurité commune et de la Commission sur le Moyen-Orient.\n      \n    \n    \n      \n      \n        &#160;&#160; . -Monsieur le Président, Mesdames et Messieurs, je tiens à vous remercier une fois de plus de m’avoir donné la possibilité de m’adresser à vous lors d’une séance plénière de cette Assemblée.\n        Je suis ici pour vous parler d’un sujet tout à fait d’actualité, à savoir le Moyen-Orient.\n        Je sais que cette question vous préoccupe tous beaucoup.\n      \n      \n        Les événements du Moyen-Orient nous inquiètent une fois de plus.\n        Hier, la commission des affaires étrangères, des droits de l’homme, de la sécurité commune et de la politique de défense a adopté le rapport rédigé par M. Menéndez del&#160;Valle.\n        Celui-ci a une nouvelle fois montré la détermination du Parle

Foncton de création du Term frequency de chaque mots par documents

In [68]:
#Fonction de calcul de TF
allDocs = {}
def Tfbydoc(index_inv):
  
  for m in index_inv.keys() :
    
    for i in range(len(index_inv[m])):
      tf= index_inv[m].count(index_inv[m][i])
      tf /= len(index_inv[m])
      aux ={}
      aux[index_inv[m][i]].add(tf)
    
    allDocs[index_inv[m]].add(aux)

In [ ]:
#test
index_inv['europarl/fr/ep-03-10-09-fr.txt'][10]

In [ ]:
#appel de la fonction de construction du dictionnaire des dictionnaires de term frequency
Tfbydoc(index_inv)

In [ ]:
#Fonction de calcul d'IDF
import math as m
IDF = {}
mot2idf = {}
def IDFforall(index):

  for m in index.keys():
    aux = m.log(len(index[m])/len(index_inv.keys()))
    mot2idf[m].add(aux)

In [ ]:
#fonction de calcul du tdidf

def tfidf(allDocs, mot2idf):
    tfidf = {}
    aux = {}
    for value1 in allDocs:
      for value2 in mot2idf:
        if value2 in allDocs[value1]:
            aux[value2] = mot2idf[value2] * allDocs[value1][value2]
        else:
            aux[value2] = 0
      tfidf[value1] = aux
  